# Task 4: Guessing the secret sequence
This notebook acts as the "hacker" trying to guess the password through the network connection.

### 1. HW initialization
Make sure `task4.bit` and `task4.hwh` are uploaded to the same directory as this notebook.

In [1]:
from pynq import Overlay
import time

# load the hardware overlay
ol = Overlay("task4.bit")

# map the gpios based on the block design
hacker_gpio = ol.axi_gpio_0
match_gpio  = ol.axi_gpio_1

### 2. Helpers
These functions handle the hardware-level communication protocol.

In [2]:
def send_hacker_bit(val):
    """
    sends a single bit to the pl.
    protocol: trigger high -> trigger low.
    """
    # trigger high (bit 1 = 1) | data bit (bit 0 = val)
    hacker_gpio.write(0, 0x2 | (val & 1)) 
    time.sleep(0.01) # short delay for hardware to catch the edge
    
    # trigger low (bit 1 = 0) | data bit (bit 0 = val)
    hacker_gpio.write(0, 0x0 | (val & 1))
    
    # read the current match count from pl
    return match_gpio.read()

def reset_game():
    print("please press btn1 (set button) on the fpga to reset the secret...")
    time.sleep(3)
    print("ready to hack!")

### 3. The brute force attack
this loop sends the bit stream and displays the correct count alongside the stream history.

In [15]:
# %% [markdown]
# ### 3. the interactive hacker
# this function tracks the exact sliding window that the fpga sees in its memory.

# %%
def smart_hack(bit_stream):
    print(f"starting attack with full stream: {bit_stream}")
    print("-" * 60)
    
    # this string acts like the 'guess_reg' in your verilog code.
    # it holds only the last 8 bits sent to the fpga.
    fpga_window = ""
    
    for i, bit in enumerate(bit_stream):
        # 1. send the bit to the hardware
        current_match = send_hacker_bit(int(bit))
        
        # 2. update our software mirror of the fpga's shift register
        fpga_window += bit
        if len(fpga_window) > 8:
            # slice off the oldest bit, keeping only the 8 most recent
            fpga_window = fpga_window[1:] 
        
        # 3. print the debug info with nice formatting
        print(f"step {i+1:02d} | sent: {bit} | fpga memory: {fpga_window:>8} | match count: {current_match}")
        
        # 4. check for success
        if current_match == 8:
            print("-" * 60)
            print("success: system unlocked! (green leds should be on)")
            return
            
    print("-" * 60)
    print("hack finished. the sequence did not unlock the system.")

# %% [markdown]
# ### 4. run the test
# put your guesses in a continuous string here.

# %%
# if your secret is '01001001', let's test a sequence with some mistakes in it
# to see the longest prefix match "slide back" logic in action.
test_stream = "0001001001" 

smart_hack(test_stream)

starting attack with full stream: 0001001001
------------------------------------------------------------
step 01 | sent: 0 | fpga memory:        0 | match count: 0
step 02 | sent: 0 | fpga memory:       00 | match count: 0
step 03 | sent: 0 | fpga memory:      000 | match count: 0
step 04 | sent: 1 | fpga memory:     0001 | match count: 1
step 05 | sent: 0 | fpga memory:    00010 | match count: 2
step 06 | sent: 0 | fpga memory:   000100 | match count: 3
step 07 | sent: 1 | fpga memory:  0001001 | match count: 1
step 08 | sent: 0 | fpga memory: 00010010 | match count: 2
step 09 | sent: 0 | fpga memory: 00100100 | match count: 3
step 10 | sent: 1 | fpga memory: 01001001 | match count: 1
------------------------------------------------------------
hack finished. the sequence did not unlock the system.
